In [1]:
from absl import app
from nasbench import api

In [2]:
INPUT = 'input'
OUTPUT = 'output'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'

In [5]:
# Load the data from file (this will take some time)
nasbench = api.NASBench('./dataset/nasbench_full.tfrecord')

# Create an Inception-like module (5x5 convolution replaced with two 3x3
# convolutions).
model_spec = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
            [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
            [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
            [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
            [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
            [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
            [0, 0, 0, 0, 0, 0, 0]],   # output layer
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT])

# Query this model from dataset, returns a dictionary containing the metrics
# associated with this model.
data = nasbench.query(model_spec)

Loading dataset from file... This may take a few minutes...


100%|██████████| 5083488/5083488 [01:59<00:00, 42649.09it/s]

Loaded dataset in 119 seconds


In [6]:
data

{'module_adjacency': array([[0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8),
 'module_operations': ['input',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'conv3x3-bn-relu',
  'conv3x3-bn-relu',
  'output'],
 'trainable_parameters': 2694282,
 'training_time': 1157.675048828125,
 'train_accuracy': 1.0,
 'validation_accuracy': 0.9378004670143127,
 'test_accuracy': 0.932692289352417}

In [8]:
nasbench.evaluate(model_spec, './test_output')

ValueError: no data files provided